In [17]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

@xr.register_dataset_accessor("shots")
class ShotAccessor:
    def __init__(self, xarray_obj):
        self._obj = xarray_obj
        self._center = None

    def to_datatree(self):
        result = self.to_dict()
        return datatree.DataTree.from_dict(result)
        
    def to_dict(self):
        offsets = self._obj.offsets.compute()
        ntime = self._obj.dims['time']

        shot_groups = {}
        lower = 0
        for shot_index, (shot, offset) in enumerate(zip(self._obj.shot_index.values, offsets.values)):
            upper = lower+offset
            shot_slice = self._obj.isel(dict(time=np.arange(lower, upper), shot_index=shot_index))
            shot_groups[shot] = shot_slice
            lower = upper

        return shot_groups

    def groupby(self):
        ntime = self._obj.dims['time']
        shot_groups = {}
        lower = 0
        offsets = self._obj.offsets.compute()
        shots = self._obj.shot_index.compute()
        index = np.empty(ntime, dtype='<U6')
        for shot_index, (shot, offset) in enumerate(zip(shots.values, offsets.values)):
            upper = lower+offset
            index[lower:upper] = shot
            lower = upper

        index = xr.DataArray(index, dims=['time'])
        return self._obj.groupby(index)


/tmp/ipykernel_741601/2139746591.py:6: AccessorRegistrationWarning: registration of accessor <class '__main__.ShotAccessor'> under name 'shots' for type <class 'xarray.core.dataset.Dataset'> is overriding a preexisting attribute with the same name.
  class ShotAccessor:


In [33]:
ds = xr.open_zarr('../data/example2.zarr')
shot_dict = ds.shots.to_dict()
s = shot_dict['28416'].data#.isel(dict(time=50))

s = s.compute()
s
# plt.matshow(s, vmin=7500)
# plt.hist(s.values.flatten(), bins=100)

<xarray.DataArray 'data' (time: 59, height: 256, width: 320)>
array([[[   0, 9305, 9371, ..., 7611, 7650, 7527],
        [   0, 9293, 9276, ..., 7680, 7668, 7629],
        [   0, 9312, 9303, ..., 7693, 7668, 7672],
        ...,
        [   0, 9181, 9212, ..., 7842, 7849, 7877],
        [   0, 9157, 9195, ..., 7845, 7866, 7881],
        [   0, 9128, 9170, ..., 7847, 7883, 7885]],

       [[   0, 9309, 9369, ..., 7611, 7650, 7533],
        [   0, 9304, 9283, ..., 7683, 7674, 7635],
        [   0, 9296, 9300, ..., 7688, 7668, 7671],
        ...,
        [   0, 9197, 9222, ..., 7855, 7865, 7887],
        [   0, 9167, 9201, ..., 7858, 7875, 7891],
        [   0, 9145, 9192, ..., 7866, 7902, 7899]],

       [[   0, 9318, 9376, ..., 7622, 7655, 7541],
        [   0, 9313, 9290, ..., 7699, 7678, 7640],
        [   0, 9314, 9317, ..., 7696, 7675, 7677],
        ...,
...
        ...,
        [   0, 9173, 9198, ..., 7835, 7852, 7870],
        [   0, 9144, 9184, ..., 7842, 7861, 7873],
        [   0, 9130, 9176, ..., 7837, 7874, 7874]],

       [[   0, 9304, 9367, ..., 7611, 7646, 7537],
        [   0, 9297, 9275, ..., 7686, 7671, 7632],
        [   0, 9304, 9301, ..., 7691, 7671, 7678],
        ...,
        [   0, 9185, 9204, ..., 7836, 7847, 7872],
        [   0, 9154, 9191, ..., 7837, 7865, 7872],
        [   0, 9132, 9164, ..., 7841, 7879, 7881]],

       [[   0, 9308, 9365, ..., 7601, 7649, 7530],
        [   0, 9301, 9284, ..., 7680, 7670, 7632],
        [   0, 9307, 9309, ..., 7692, 7670, 7673],
        ...,
        [   0, 9193, 9218, ..., 7858, 7867, 7886],
        [   0, 9162, 9201, ..., 7856, 7872, 7877],
        [   0, 9143, 9184, ..., 7859, 7899, 7886]]], dtype=uint16)
Coordinates:
    shot_index  <U5 '28416'
  * time        (time) float64 -0.04754 -0.04437 -0.0412 ... 0.1299 0.133 0.1362
Dimensions without coordinates: height, width